In [1]:
import pandas as pd
from gensim.models import word2vec
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stops = stopwords.words('english')
from string import punctuation

ls   = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
ls[:5]

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


,ID,review,sentiment
0,41411,I watched this film because I'm a big fan of R...,0
1,37586,It does not seem that this movie managed to pl...,1
2,6017,"Enough is not a bad movie , just mediocre .",0
3,44656,my friend and i rented this one a few nights a...,0
4,38711,"Just about everything in this movie is wrong, ...",0


# 清除標點符號、切割小寫、stopword

In [ ]:
remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'

Sentense = []
for r in ls.review:
    string = re.sub(remove_chars, " ",r)                                                         # 去除標點符號
    tokens = nltk.word_tokenize(string.lower())                                                  # 句子切割和小寫
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens if t not in stops]                 # 去除stopword，lemmatize則是字根化
    Sentense.append(tokens)

# 測試集資料

In [ ]:
T_Sentense = []
for r in test.review:
    string = re.sub(remove_chars, " ",r)                                                         # 去除標點符號
    tokens = nltk.word_tokenize(string.lower())                                                  # 句子切割和小寫
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens if t not in stops]                 # 去除stopword，lemmatize則是字根化
    T_Sentense.append(tokens)

# W2V

In [ ]:
vector_size = 128     #每個字的向量維度
model = word2vec.Word2Vec(Sentense+T_Sentense, min_count=1, vector_size=vector_size, window=10)

In [ ]:
X = []
for se in Sentense:
    zero = np.zeros(vector_size)
    for word in se:
        zero += model.wv[word]
    
    X.append(zero)

In [ ]:
X = np.array(X)
Y = np.array(ls.sentiment)
len(Y)
len(X)

# 貝氏分類

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) # 70% training and 30% test

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#Create a Gaussian Classifier
gnb = LinearSVC()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# 測試集結果

In [ ]:
Z = []
for se in T_Sentense:
    zero = np.zeros(vector_size)
    for word in se:
        zero += model.wv[word]
    Z.append(zero)

In [ ]:
Z = scaler.fit_transform(Z)
result = gnb.predict(Z)

In [ ]:
pd.DataFrame({"ID":test.ID,"sentiment":result}).to_csv("result.csv",index=False)

# 想測試TFIDF  
### 維度會太大，須限制維度
### TFIDF完，維度就太大

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer


remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'

Sentense = []
for r in ls.review:
    s = " "
    string = re.sub(remove_chars, " ", r)                                                         # 去除標點符號
    tokens = nltk.word_tokenize(string.lower())                                                  # 句子切割和小寫
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens if t not in stops]                 # 去除stopword，lemmatize則是字根化
    corpus = s.join(tokens)
    Sentense.append(corpus)


T_Sentense = []
for r in test.review:
    t = " "
    string = re.sub(remove_chars, " ",r)                                                         # 去除標點符號
    tokens = nltk.word_tokenize(string.lower())                                                  # 句子切割和小寫
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens if t not in stops]                 # 去除stopword，lemmatize則是字根化
    corpus = t.join(tokens)
    T_Sentense.append(corpus)


vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(Sentense+T_Sentense)



In [7]:
Z = X.toarray()[29341:]
X = X.toarray()[:29341]
Y = np.array(ls.sentiment)
print(len(X))
print(len(Y))

29341
29341


In [8]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3) # 70% training and 30% test

In [9]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#Create a Gaussian Classifier
gnb = LinearSVC()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [10]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7946154719981824


In [11]:
Z = scaler.fit_transform(Z)
result = gnb.predict(Z)

In [12]:
pd.DataFrame({"ID":test.ID,"sentiment":result}).to_csv("result.csv",index=False)

In [ ]:
XGBClassifier(
        #樹的個數
        n_estimators=100,
        # 如同學習率
        learning_rate= 0.3, 
        # 構建樹的深度，越大越容易過擬合    
        max_depth=6, 
        # 隨機取樣訓練樣本 訓練例項的子取樣比
        subsample=1, 
        # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        gamma=0, 
        # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        reg_lambda=1,  
        
        #最大增量步長，我們允許每個樹的權重估計。
        max_delta_step=0,
        # 生成樹時進行的列取樣 
        colsample_bytree=1, 

        # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
        # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
        #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        min_child_weight=1, 

        #隨機種子
        seed=1000 
        
        # L1 正則項引數
#        reg_alpha=0,
        
        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #scale_pos_weight=1,
        
        #多分類的問題 指定學習任務和相應的學習目標
        #objective= 'multi:softmax', 
        
        # 類別數，多分類與 multisoftmax 並用
        #num_class=10,
        
        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        silent=0 ,
        # cpu 執行緒數 預設最大
#        nthread=4,
    
        #eval_metric= 'auc'
)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
X, y = make_classification(n_features=4, random_state=0)
clf = make_pipeline(StandardScaler(),
                    LinearSVC(random_state=0, tol=1e-5))
clf.fit(X, y)

In [ ]:
y